# Homework 1: Intro to Deep RL with Single Agent Training Environments

The goal of this assignment is to gain hands-on experience with the key components of Reinforcement Learning (RL) environments. 

For more details please checkout [HW1.md](../HW1.md)

## Setup

You will need to make a copy of this notebook in your Google Drive before you can edit the homework files. You can do so with **File &rarr; Save a copy in Drive**.

In [ ]:
#@title Mount Your Google Drive
#@markdown Your work will be stored in a folder called `cs285_f2022` by default to prevent Colab instance timeouts from deleting your edits.

import os
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title Setup Mount Symlink

DRIVE_PATH = '/content/gdrive/My\ Drive/rl_class'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
SYM_PATH = '/content/rl_class'
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

In [ ]:
#@title Apt Install Requirements

#@markdown Run each section with Shift+Enter

#@markdown Double-click on section headers to show code.

!apt update
!apt install -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python-opengl \
        ffmpeg

In [ ]:
#@title Clone Homework Repo

%cd $SYM_PATH
# !git clone https://github.com/heng2j/multigrid.git
%cd multigrid/
%pip install -r requirements_colab.txt
%pip install -e .

In [ ]:
!pip install gym tensorboard moviepy torch opencv-python swig box2d-py ray[rllib] scikit-image pygame numba Gymnasium black PyYAML

## Editing Code

To edit code, click the folder icon on the left menu. Navigate to the corresponding file (`multigrid/...`). Double click a file to open an editor. There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window). We sync your edits to Google Drive so that you won't lose your work in the event of an instance timeout, but you will need to re-mount your Google Drive and re-install packages with every new instance.

In [ ]:
#@title Imports
from __future__ import annotations

import json
import pathlib
import os
import numpy as np
from dataclasses import dataclass, asdict, field
from types import SimpleNamespace
import git
from IPython.display import Image

import ray
from ray import tune
from ray.rllib.algorithms import AlgorithmConfig, Algorithm
from ray.tune import CLIReporter
from ray.air.integrations.mlflow import MLflowLoggerCallback


from multigrid.envs import *
from multigrid.utils.training_utilis import algorithm_config, can_use_gpu,  get_checkpoint_dir, policy_mapping_fn





In [ ]:
#@title Fix Variables
SCRIPT_PATH = str(pathlib.Path(__file__).parent.absolute().parent.absolute())

## For Agent Training



In [ ]:
#@title Configurable Training Function

# Set up Ray CLIReporter 
# NOTE Limit the number of rows.
reporter = CLIReporter(max_progress_rows=10)

# Configurable Training Function
def train(
    algo: str,
    config: AlgorithmConfig,
    stop_conditions: dict,
    save_dir: str,
    user_name: str,
    checkpoint_freq: int = 20,
    load_dir: str | None = None,
    local_mode: bool = False,
    experiment_name: str = "testing_experiment",
    mlflow_tracking_uri: str = "submission/mlflow",
):
    """
    Train an RLlib algorithm.

    Parameters
    ----------
    algo : str
        Name of the RLlib-registered algorithm to use.
    config : AlgorithmConfig
        Algorithm-specific configuration parameters.
    stop_conditions : dict
        Conditions to stop the training loop.
    save_dir : str
        Directory to save training checkpoints and results.
    user_name : str
        Experimenter's name.
    checkpoint_freq : int, optional
        Frequency of saving checkpoints, by default 20.
    load_dir : str, optional
        Directory to load pre-trained models from, by default None.
    local_mode : bool, optional
        Set to True to run Ray in local mode for debugging, by default False.
    experiment_name : str, optional
        Name of the experiment, by default "testing_experiment".
    mlflow_tracking_uri : str, optional
        Directory to save MLFlow metrics and artifacts, by default "submission/mlflow".
    """

    # Initialize Ray.
    ray.init(num_cpus=(config.num_rollout_workers + 1), local_mode=local_mode)

    # Execute training
    tune.run(
        algo,
        stop=stop_conditions,
        config=config,
        local_dir=save_dir,
        verbose=1,
        restore=get_checkpoint_dir(load_dir),
        checkpoint_freq=checkpoint_freq,
        checkpoint_at_end=True,
        progress_reporter=reporter,
        callbacks=[
            MLflowLoggerCallback(
                tracking_uri=mlflow_tracking_uri,
                experiment_name=experiment_name,
                tags={
                    "user_name": user_name,
                    "git_commit_hash": git.Repo(SCRIPT_PATH).head.commit,
                },
                save_artifact=True,
            )
        ],
    )

    # Shutdown Ray after training is complete
    ray.shutdown()




In [ ]:
#@title Training Arguments
@dataclass
class Args:

  #@markdown agent config
  num_agents: int = 1 #@param {type: "integer"}
  algo: str = "PPO"  #@param {type: "string"}
  framework: str = "torch" #@param ['torch', 'tf2']

  #@markdown environemnt config
  env: str = "MultiGrid-CompetativeRedBlueDoor-v0"  #@param {type: "string"}

  #@markdown training config
  num_workers: int = 10  #@param {type: "integer"}
  num_gpus: int = 0 #@param {type: "integer"}
  # Please only keep the checkpoints that you want to submit
  save_dir: str = "../../submission/ray_results/" #@param {type: "string"}
  user_name: str = "<Your Name>" #@param {type: "string"}
  experiment_name: str = "testing_experiment", #@param {type: "string"}
  mlflow_tracking_uri: str = "../../submission/mlflow/", #@param {type: "string"}
  checkpoint_freq: int = 20 #@param {type: "integer"}
  num_timesteps: float = 5e5 #@param {type: "string"}
  checkpoint_freq: int = 20 #@param {type: "integer"}
  seed: int = 1 #@param {type: "integer"}






In [ ]:
#@title Set up Training Arguments
def to_namespace(self):
    return SimpleNamespace(**asdict(self))

args = Args()
print(args)  # Prints the values of all attributes

args = args.to_namespace()
print(args)

config = algorithm_config(**vars(args))
config.seed = args.seed
stop_conditions = {'timesteps_total': args.num_timesteps}

print(config.env)
print(config.seed)

 ## Initialize and Show Tensorboard Before Training

 Filter tags for key performance metrics:

episode_len_mean|ray/tune/episode_reward_mean|episode_reward_min|entropy|vf|loss|kl|cpu|ram

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Start TensorBoard and Map the `logdir`` to `save_dir` i.e. `/content/rl_class/multigrid/submission/ray_results/PPO`
%tensorboard --logdir /content/rl_class/multigrid/submission/ray_results/PPO

## Execute training

In [ ]:
#@title Execute Training
train(args.algo, config, stop_conditions, args.save_dir, None)

# Please remember to clear your training outputs before you submit your notebook to reduce file size and increase readability

In [ ]:
# NOTE Manually shutdown Ray if needed
ray.shutdown()

## Submission for Task 3 - Monitor and Track Agent Training with Tensorboard and Save Out Visualization from Evaluation

1. Please take screenshots of your Tensorboard plots that highlight your performance metrics
2. Embedd your images here in CoLab
3. Only save the best checkpoint and video in the /submission folder and push to your repo


In [ ]:
from google.colab import files
from IPython.display import Image

uploaded = files.upload()

# Assuming a single image file is uploaded
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  display(Image(fn))

In [ ]:
#@title Your Tensorboard Screenshots Go Here
...

## Agent Evaluation

In [ ]:

#@title Evaluation Arguments

@dataclass
class EvalArgs:

  #@markdown agent config
  num_agents: int = 1 #@param {type: "integer"}
  algo: str = "PPO"  #@param {type: "string"}
  framework: str = "torch" #@param ['torch', 'tf2']
  lstm: bool = False #@param {type: "boolean"}

  #@markdown environemnt config
  env: str = "MultiGrid-CompetativeRedBlueDoor-v0"  #@param {type: "string"}
  env_config: json.load = {}

  #@markdown Evaluation config
  num_episodes: int = 10 #@param {type: "integer"}
  load_dir: str = "../../submission/ray_results/PPO/PPO_MultiGrid-CompetativeRedBlueDoor-v2_de334_00000_0_2023-07-14_17-32-00" #@param {type: "string"}
  gif: str = "../../submission/notebook/result.gif" #@param {type: "string"}
  # num_timesteps: float = 5e5 #@param {type: "string"}
  # ep_len: int = 1000 #@param {type: "integer"}
  # seed: int = 1 #@param {type: "integer"}
  env_config: dict = field(default_factory=dict)

  def to_namespace(self):
    return SimpleNamespace(**asdict(self))







In [ ]:
#@title Configurable Evaluation Function


def visualize(algorithm: Algorithm, num_episodes: int = 100) -> list[np.ndarray]:
    """
    Visualize trajectories from trained agents.

    Parameters
    ----------
    algorithm : Algorithm
        The algorithm instance, from which the policy is derived.
    num_episodes : int, optional
        The number of episodes to visualize, by default 100.

    Returns
    -------
    list[np.ndarray]
        A list of frames depicting the agents' trajectories.
    """
    # An empty list to store frames
    frames = []

    # Create an environment instance using the environment creator from the algorithm
    env = algorithm.env_creator(algorithm.config.env_config)

    # Iterate over the defined number of episodes
    for episode in range(num_episodes):
        print("\n", "-" * 32, "\n", "Episode", episode, "\n", "-" * 32)

        # Initialize a dictionary to store episode rewards for each agent
        episode_rewards = {agent_id: 0.0 for agent_id in env.get_agent_ids()}
        # Initialize termination and truncation flags for the episode
        terminations, truncations = {"__all__": False}, {"__all__": False}
        # Reset the environment to get initial observations and info
        observations, infos = env.reset()
        # Initialize the states for each agent
        states = {
            agent_id: algorithm.get_policy(policy_mapping_fn(agent_id)).get_initial_state()
            for agent_id in env.get_agent_ids()
        }

        # Continue the episode until a termination or truncation condition is met
        while not terminations["__all__"] and not truncations["__all__"]:
            # Append the current environment frame to the list of frames
            frames.append(env.get_frame())

            actions = {}
            for agent_id in env.get_agent_ids():
                # Single-agent
                actions[agent_id] = algorithm.compute_single_action(
                    observations[agent_id],
                    states[agent_id],
                    policy_id=policy_mapping_fn(agent_id),
                )

            # Perform a step in the environment using the actions
            observations, rewards, terminations, truncations, infos = env.step(actions)
            for agent_id in rewards:
                episode_rewards[agent_id] += rewards[agent_id]

        # Append the final frame of the episode
        frames.append(env.get_frame())
        print("Rewards:", episode_rewards)

    env.close()
    return frames


In [ ]:
#@title Set up Evaluation Arguments
eval_args = EvalArgs()
print(eval_args)  # Prints the values of all attributes

eval_args = eval_args.to_namespace()
print(eval_args)

eval_args.env_config.update(render_mode='human')
config = algorithm_config(
    **vars(eval_args),
    num_workers=0,
    num_gpus=0,
)

print(config.env)

algorithm = config.build()

checkpoint = get_checkpoint_dir(eval_args.load_dir)
if checkpoint:
    print(f"Loading checkpoint from {checkpoint}")
    algorithm.restore(checkpoint)

In [ ]:
#@title Execute Evaluation
frames = visualize(algorithm, num_episodes=eval_args.num_episodes)


In [ ]:
#@title Visualize and Display Evaluated Agent Behavniors
if eval_args.gif:
    import imageio
    filename = eval_args.gif if eval_args.gif.endswith('.gif') else f'{eval_args.gif}.gif'
    print(f"Saving GIF to {filename}")
    # write to file
    imageio.mimsave(filename, frames)


# Load the GIF
Image(filename=filename)
display(Image(filename=filename))